In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.12:0.14.0 pyspark-shell'

import re
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, lower, explode
from pyspark.sql.types import StringType, ArrayType


In [2]:
# Create Spark Session from Builder
spark = SparkSession.builder.getOrCreate()


:: loading settings :: url = jar:file:/usr/lib/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.databricks#spark-xml_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e3ff2b89-7c45-429e-aecc-3588f482e54d;1.0
	confs: [default]
	found com.databricks#spark-xml_2.12;0.14.0 in central
	found commons-io#commons-io;2.8.0 in central
	found org.glassfish.jaxb#txw2;2.3.4 in central
	found org.apache.ws.xmlschema#xmlschema-core;2.2.5 in central
:: resolution report :: resolve 398ms :: artifacts dl 9ms
	:: modules in use:
	com.databricks#spark-xml_2.12;0.14.0 from central in [default]
	commons-io#commons-io;2.8.0 from central in [default]
	org.apache.ws.xmlschema#xmlschema-core;2.2.5 from central in [default]
	org.glassfish.jaxb#txw2;2.3.4 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|

In [3]:
# input file
fp = 'hdfs:/wiki-small.xml'
# create a DataFrame
df = spark.read.format('xml').options(rowTag='page').load(fp)
# print schema
df.printSchema()

root
 |-- id: long (nullable = true)
 |-- ns: long (nullable = true)
 |-- redirect: struct (nullable = true)
 |    |-- _VALUE: string (nullable = true)
 |    |-- _title: string (nullable = true)
 |-- revision: struct (nullable = true)
 |    |-- comment: string (nullable = true)
 |    |-- contributor: struct (nullable = true)
 |    |    |-- id: long (nullable = true)
 |    |    |-- ip: string (nullable = true)
 |    |    |-- username: string (nullable = true)
 |    |-- format: string (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- minor: string (nullable = true)
 |    |-- model: string (nullable = true)
 |    |-- parentid: long (nullable = true)
 |    |-- sha1: string (nullable = true)
 |    |-- text: struct (nullable = true)
 |    |    |-- _VALUE: string (nullable = true)
 |    |    |-- _bytes: long (nullable = true)
 |    |    |-- _xml:space: string (nullable = true)
 |    |-- timestamp: timestamp (nullable = true)
 |-- title: string (nullable = true)



In [4]:
df1 = df.withColumn("text", lower(col("revision.text._VALUE"))).withColumn("title", lower(col("title"))) \
.select("title", "text")
df1.show()

+--------------------+--------------------+
|               title|                text|
+--------------------+--------------------+
| accessiblecomputing|#redirect [[compu...|
|           anarchism|{{short descripti...|
|  afghanistanhistory|#redirect [[histo...|
|afghanistangeography|#redirect [[geogr...|
|   afghanistanpeople|#redirect [[demog...|
|afghanistancommun...|#redirect [[commu...|
|afghanistantransp...|#redirect [[trans...|
| afghanistanmilitary|#redirect [[afgha...|
|afghanistantransn...|#redirect [[forei...|
| assistivetechnology|#redirect [[assis...|
|        amoeboidtaxa|#redirect [[amoeb...|
|              autism|#redirect [[autis...|
|      albaniahistory|#redirect [[histo...|
|       albaniapeople|#redirect [[demog...|
|        aswemaythink|#redirect [[as_we...|
|   albaniagovernment|#redirect [[polit...|
|      albaniaeconomy|#redirect [[econo...|
|              albedo|{{short descripti...|
|afroasiaticlanguages|#redirect [[afroa...|
|  artificallanguages|#redirect 

In [5]:
# an UDF to extract links from "revision" columns and return a list of 
def extract_links(text):
    # this regex handles all situations needed except the all whitespace or empty string case
    pattern = '(?<=\[\[)((?:category:)[^#\|\]]*?|[^#\|\]:]*?)(?=\||\]\])'
    links = re.findall(pattern, str(text), flags=re.I | re.M)
    # filter out empty strings and strings with all whitespaces
    non_empty = [l for l in links if not l.isspace() and len(l)>0]
    return non_empty

extract_links_udf = udf(extract_links, ArrayType(StringType()))


In [6]:
# REGEX testing
# text = "[[    ]][[]][[File:Hercules Musei Capitolini MC1265 n2.jpg|thumb|right|upright|[[Heracles]] with the apple of [[Hesperides]]]]"
# pattern = '(?<=\[\[)((?:category:)[^#\|\]]*?|[^#\|\]:]*?)(?=\||\]\])'
# links = re.findall(pattern, text, flags=re.I | re.M)

# print(links)

In [7]:
# Explode the links list
df2 = df1.withColumn("links", extract_links_udf(col("text"))).withColumn("link", explode("links")).drop("links", "text")
# Sort two columns in Ascending order, and limit to 10 rows
df3 = df2.sort("title", "link").limit(10)
df3.show()



+--------------------+--------------------+
|               title|                link|
+--------------------+--------------------+
|"hello, world!" p...|                .deb|
|"hello, world!" p...|3d computer graphics|
|"hello, world!" p...|ada (programming ...|
|"hello, world!" p...|            algol 60|
|"hello, world!" p...|application progr...|
|"hello, world!" p...|               ascii|
|"hello, world!" p...|b (programming la...|
|"hello, world!" p...|               basic|
|"hello, world!" p...|                bcpl|
|"hello, world!" p...|           bell labs|
+--------------------+--------------------+



In [8]:
df3.write.mode("overwrite").csv("gs://4121-programming2/notebooks/jupyter/task2.csv",header=False,sep='\t')

----------------------------------------                                        
Exception happened during processing of request from ('127.0.0.1', 49892)
Traceback (most recent call last):
  File "/opt/conda/miniconda3/lib/python3.8/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/conda/miniconda3/lib/python3.8/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/opt/conda/miniconda3/lib/python3.8/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/conda/miniconda3/lib/python3.8/socketserver.py", line 747, in __init__
    self.handle()
  File "/usr/lib/spark/python/pyspark/accumulators.py", line 262, in handle
    poll(accum_updates)
  File "/usr/lib/spark/python/pyspark/accumulators.py", line 235, in poll
    if func():
  File "/usr/lib/spark/python/pyspark/accumulators.py", line 239, in accum_updat